# Emergence of Latent Binary Encoding in Deep Neural Network Classifiers
---

### Supplementary material to reproduce results

All plots shown in the manuscript can be made using this notebook. Results are loaded from two 'best_results.pkl' files which contain data relative to the CIFAR10 and CIFAR100 dataset, correspondingly. 
Each experiment consists of running multiple trainings for different learning rates, and the training which has the best accuracy on the training set at the last epoch is picked. Experiments can be run on a GPU cluster using a Slurm script that we provide in 'emergence_binary_encoding/scripts.sh'. Best results for each experiment can then be selected using the Python script that we provide in 'emergence_binary_encoding/find_best_results.py'

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import multivariate_normal
import pickle
from sklearn.mixture import GaussianMixture
# from scipy.stats import norm
# import scipy
import torch
import torch.nn as nn
# from scipy.spatial import distance
import seaborn as sns
import yaml
import glob
import os
import matplotlib
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture


In [ ]:
dataset_dir = '../results/'
with open(dataset_dir + '/svhn/best_results.pkl', 'rb') as file:
    results_svhn = pickle.load(file)
    
dataset_dir = '../results_plot'
with open(dataset_dir + '/cifar10/best_results.pkl', 'rb') as file:
    results_10 = pickle.load(file)

dataset_dir = '../results_plot'
with open(dataset_dir + '/cifar100/best_results.pkl', 'rb') as file:
    results_100 = pickle.load(file)


In [ ]:
epochs = results_100['bin_enc']['training_hypers']['epochs'][0]
logging = results_100['bin_enc']['training_hypers']['logging'][0]

In [ ]:
sns.set(style="whitegrid")
alpha=0.3

In [ ]:
models = [ 'bin_enc', '512_bin_enc', 'no_pen', 'dropout_no_pen', 'lin_pen', 'dropout_lin_pen', 'nonlin_pen']

In [ ]:
color_bin = '#1f77b4'
color_lin = '#2ca02c'
color_no = '#ff7f0e'
color_nonlin = '#d62728'
color_dropout_no = '#9467bd'
color_bin_wide = '#8c564b' 
color_dropout_lin = '#e377c2'
label_size = 18
legend_size = 12

In [ ]:
for model in models:
    print(model, results_svhn[model]['convergence_epoch'])

In [ ]:
for model in models:
    print(model, results_10[model]['convergence_epoch'])

In [ ]:
for model in models:
    print(model, results_100[model]['convergence_epoch'])

### Figure 1

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(10, 8), sharex=True)

color_0 = color_bin
color_1 = color_lin
color_2 = color_bin_wide
color_3 = color_dropout_lin



x = np.arange(logging, epochs+logging,logging)

y = np.mean(results_svhn['bin_enc']['binarity_train']['score'], axis=1)
std = np.std(results_svhn['bin_enc']['binarity_train']['score'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[0][0], color=color_0, label='BinEnc')
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_0)

y = np.mean(results_svhn['512_bin_enc']['binarity_train']['score'], axis=1)
std = np.std(results_svhn['512_bin_enc']['binarity_train']['score'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[0][0], color=color_2, label='WideBinEnc')
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_2)

y = np.mean(results_svhn['lin_pen']['binarity_train']['score'], axis=1)
std = np.std(results_svhn['lin_pen']['binarity_train']['score'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[0][0], color=color_1, label='LinPen')
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_1)


y = np.mean(results_svhn['dropout_lin_pen']['binarity_train']['score'], axis=1)
std = np.std(results_svhn['dropout_lin_pen']['binarity_train']['score'], axis=1)
sns.lineplot(x=x, y=y, marker='o',  linestyle='--', ax=axes[0][0], color=color_3, label='DropoutLinPen')
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_3)

y = np.mean(results_svhn['bin_enc']['binarity_train']['stds'], axis=1)
std = np.std(results_svhn['bin_enc']['binarity_train']['stds'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[1][0], color=color_0)
axes[1][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_0)

y = np.mean(results_svhn['lin_pen']['binarity_train']['stds'], axis=1)
std = np.std(results_svhn['lin_pen']['binarity_train']['stds'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[1][0], color=color_1)
axes[1][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_1)

y = np.mean(results_svhn['512_bin_enc']['binarity_train']['stds'], axis=1)
std = np.std(results_svhn['512_bin_enc']['binarity_train']['stds'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[1][0], color=color_2, label='WideBinEnc')
axes[1][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_2)

y = np.mean(results_svhn['dropout_lin_pen']['binarity_train']['stds'], axis=1)
std = np.std(results_svhn['dropout_lin_pen']['binarity_train']['stds'], axis=1)
sns.lineplot(x=x, y=y, marker='o',  linestyle='--', ax=axes[1][0], color=color_3, label='DropoutLinPen')
axes[1][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_3)

y = np.mean(results_svhn['bin_enc']['binarity_train']['peaks_distance_mean'], axis=1)
std = np.std(results_svhn['bin_enc']['binarity_train']['peaks_distance_mean'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[2][0], color=color_0, label='BinEnc')
axes[2][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_0)

y = np.mean(results_svhn['lin_pen']['binarity_train']['peaks_distance_mean'], axis=1)
std = np.std(results_svhn['lin_pen']['binarity_train']['peaks_distance_mean'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[2][0], color=color_1, label='LinPen')
axes[2][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_1)

y = np.mean(results_svhn['512_bin_enc']['binarity_train']['peaks_distance_mean'], axis=1)
std = np.std(results_svhn['512_bin_enc']['binarity_train']['peaks_distance_mean'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[2][0], color=color_2, label='WideBinEnc')
axes[2][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_2)

y = np.mean(results_svhn['dropout_lin_pen']['binarity_train']['peaks_distance_mean'], axis=1)
std = np.std(results_svhn['dropout_lin_pen']['binarity_train']['peaks_distance_mean'], axis=1)
sns.lineplot(x=x, y=y, marker='o',  linestyle='--', ax=axes[2][0], color=color_3, label='DropoutLinPen')
axes[2][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_3)



y = np.mean(results_10['bin_enc']['binarity_train']['score'], axis=1)
std = np.std(results_10['bin_enc']['binarity_train']['score'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[0][1], color=color_0, label='BinEnc')
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_0)

y = np.mean(results_10['lin_pen']['binarity_train']['score'], axis=1)
std = np.std(results_10['lin_pen']['binarity_train']['score'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[0][1], color=color_1, label='LinPen')
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_1)

y = np.mean(results_10['512_bin_enc']['binarity_train']['score'], axis=1)
std = np.std(results_10['512_bin_enc']['binarity_train']['score'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[0][1], color=color_2, label='WideBinEnc')
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_2)

y = np.mean(results_10['dropout_lin_pen']['binarity_train']['score'], axis=1)
std = np.std(results_10['dropout_lin_pen']['binarity_train']['score'], axis=1)
sns.lineplot(x=x, y=y, marker='o',  linestyle='--', ax=axes[0][1], color=color_3, label='DropoutLinPen')
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_3)

y = np.mean(results_10['bin_enc']['binarity_train']['stds'], axis=1)
std = np.std(results_10['bin_enc']['binarity_train']['stds'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[1][1], color=color_0)
axes[1][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_0)

y = np.mean(results_10['lin_pen']['binarity_train']['stds'], axis=1)
std = np.std(results_10['lin_pen']['binarity_train']['stds'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[1][1], color=color_1)
axes[1][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_1)

y = np.mean(results_10['512_bin_enc']['binarity_train']['stds'], axis=1)
std = np.std(results_10['512_bin_enc']['binarity_train']['stds'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[1][1], color=color_2, label='WideBinEnc')
axes[1][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_2)

y = np.mean(results_10['dropout_lin_pen']['binarity_train']['stds'], axis=1)
std = np.std(results_10['dropout_lin_pen']['binarity_train']['stds'], axis=1)
sns.lineplot(x=x, y=y, marker='o',  linestyle='--', ax=axes[1][1], color=color_3, label='DropoutLinPen')
axes[1][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_3)

y = np.mean(results_10['bin_enc']['binarity_train']['peaks_distance_mean'], axis=1)
std = np.std(results_10['bin_enc']['binarity_train']['peaks_distance_mean'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[2][1], color=color_0)
axes[2][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_0)

y = np.mean(results_10['lin_pen']['binarity_train']['peaks_distance_mean'], axis=1)
std = np.std(results_10['lin_pen']['binarity_train']['peaks_distance_mean'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[2][1], color=color_1)
axes[2][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_1)

y = np.mean(results_10['512_bin_enc']['binarity_train']['peaks_distance_mean'], axis=1)
std = np.std(results_10['512_bin_enc']['binarity_train']['peaks_distance_mean'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[2][1], color=color_2, label='WideBinEnc')
axes[2][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_2)

y = np.mean(results_10['dropout_lin_pen']['binarity_train']['peaks_distance_mean'], axis=1)
std = np.std(results_10['dropout_lin_pen']['binarity_train']['peaks_distance_mean'], axis=1)
sns.lineplot(x=x, y=y, marker='o',  linestyle='--', ax=axes[2][1], color=color_3, label='DropoutLinPen')
axes[2][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_3)



y = np.mean(results_100['bin_enc']['binarity_train']['score'], axis=1)
std = np.std(results_100['bin_enc']['binarity_train']['score'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[0][2], color=color_0, label='BinEnc')
axes[0][2].fill_between(x, y - std, y + std, alpha=alpha, color=color_0)

y = np.mean(results_100['lin_pen']['binarity_train']['score'], axis=1)
std = np.std(results_100['lin_pen']['binarity_train']['score'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[0][2], color=color_1, label='LinPen')
axes[0][2].fill_between(x, y - std, y + std, alpha=alpha, color=color_1)

y = np.mean(results_100['512_bin_enc']['binarity_train']['score'], axis=1)
std = np.std(results_100['512_bin_enc']['binarity_train']['score'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[0][2], color=color_2, label='WideBinEnc')
axes[0][2].fill_between(x, y - std, y + std, alpha=alpha, color=color_2)

y = np.mean(results_100['dropout_lin_pen']['binarity_train']['score'], axis=1)
std = np.std(results_100['dropout_lin_pen']['binarity_train']['score'], axis=1)
sns.lineplot(x=x, y=y, marker='o',  linestyle='--', ax=axes[0][2], color=color_3, label='DropoutLinPen')
axes[0][2].fill_between(x, y - std, y + std, alpha=alpha, color=color_3)


y = np.mean(results_100['bin_enc']['binarity_train']['stds'], axis=1)
std = np.std(results_100['bin_enc']['binarity_train']['stds'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[1][2], color=color_0)
axes[1][2].fill_between(x, y - std, y + std, alpha=alpha, color=color_0)

y = np.mean(results_100['lin_pen']['binarity_train']['stds'], axis=1)
std = np.std(results_100['lin_pen']['binarity_train']['stds'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[1][2], color=color_1)
axes[1][2].fill_between(x, y - std, y + std, alpha=alpha, color=color_1)

y = np.mean(results_100['512_bin_enc']['binarity_train']['stds'], axis=1)
std = np.std(results_100['512_bin_enc']['binarity_train']['stds'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[1][2], color=color_2, label='WideBinEnc')
axes[1][2].fill_between(x, y - std, y + std, alpha=alpha, color=color_2)

y = np.mean(results_100['dropout_lin_pen']['binarity_train']['stds'], axis=1)
std = np.std(results_100['dropout_lin_pen']['binarity_train']['stds'], axis=1)
sns.lineplot(x=x, y=y, marker='o',  linestyle='--', ax=axes[1][2], color=color_3, label='DropoutLinPen')
axes[1][2].fill_between(x, y - std, y + std, alpha=alpha, color=color_3)

y = np.mean(results_100['bin_enc']['binarity_train']['peaks_distance_mean'], axis=1)
std = np.std(results_100['bin_enc']['binarity_train']['peaks_distance_mean'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[2][2], color=color_0)
axes[2][2].fill_between(x, y - std, y + std, alpha=alpha, color=color_0)

y = np.mean(results_100['lin_pen']['binarity_train']['peaks_distance_mean'], axis=1)
std = np.std(results_100['lin_pen']['binarity_train']['peaks_distance_mean'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[2][2], color=color_1, )
axes[2][2].fill_between(x, y - std, y + std, alpha=alpha, color=color_1)

y = np.mean(results_100['512_bin_enc']['binarity_train']['peaks_distance_mean'], axis=1)
std = np.std(results_100['512_bin_enc']['binarity_train']['peaks_distance_mean'], axis=1)
sns.lineplot(x=x, y=y, marker='o', ax=axes[2][2], color=color_2, label='WideBinEnc')
axes[2][2].fill_between(x, y - std, y + std, alpha=alpha, color=color_2)

y = np.mean(results_100['dropout_lin_pen']['binarity_train']['peaks_distance_mean'], axis=1)
std = np.std(results_100['dropout_lin_pen']['binarity_train']['peaks_distance_mean'], axis=1)
sns.lineplot(x=x, y=y, marker='o',  linestyle='--', ax=axes[2][2], color=color_3, label='DropoutLinPen')
axes[2][2].fill_between(x, y - std, y + std, alpha=alpha, color=color_3)



axes[0][0].set_title('SVHN', size=label_size)
axes[0][1].set_title('CIFAR 10', size=label_size)
axes[0][2].set_title('CIFAR 100', size=label_size)

axes[2][0].set_xlabel('Epochs', size=label_size)
axes[2][1].set_xlabel('Epochs', size=label_size)
axes[2][2].set_xlabel('Epochs', size=label_size)
axes[0][0].set_ylabel('Score', size=label_size)
axes[1][0].set_ylabel('Std', size=label_size)
axes[2][0].set_ylabel('Peaks distance', size=label_size)


axes[0][0].set_ylim(-1.5,1.5)
axes[0][1].set_ylim(-1.5,1.5)
axes[0][2].set_ylim(-1.5,1.5)
axes[1][0].set_ylim(0,1)
axes[1][1].set_ylim(0,1)
axes[1][2].set_ylim(0,1)
axes[2][0].set_ylim(0,30)
axes[2][1].set_ylim(0,60)
axes[2][2].set_ylim(0,40)

for ax in axes.flat:
    ax.get_legend().remove()
axes[0][0].legend(loc='upper left', fontsize=legend_size)


plt.tight_layout()
plt.savefig('./score.png')

### Figure 2

In [ ]:
convergence_svhn = {}
convergence_10 = {}
convergence_100 = {}

for model in models:
    convergence_svhn [model] = np.mean(results_svhn[model]['convergence_epoch']) 
    convergence_10 [model] = np.mean(results_10[model]['convergence_epoch'])    
    convergence_100 [model] = np.mean(results_100[model]['convergence_epoch'])

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10, 4),  sharey=True)

label_size = 18
legend_size = 12

x = np.arange(logging, epochs+logging,logging)

y = np.mean(results_svhn['bin_enc']['same_encoding_fraction_train'], axis=1)
std = np.std(results_svhn['bin_enc']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[0], color=color_bin)
axes[0].fill_between(x, y - std, y + std, alpha=alpha, color=color_bin)

y = np.mean(results_svhn['lin_pen']['same_encoding_fraction_train'], axis=1)
std = np.std(results_svhn['lin_pen']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', color=color_lin, ax=axes[0])
axes[0].fill_between(x, y - std, y + std, alpha=alpha, color=color_lin)

y = np.mean(results_svhn['nonlin_pen']['same_encoding_fraction_train'], axis=1)
std = np.std(results_svhn['nonlin_pen']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker=x = np.arange(logging, epochs+logging,logging)

fig, axes = plt.subplots(3, 3, figsize=(10, 8), sharex=True)


y = np.mean(results_svhn['lin_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_svhn['lin_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', ax=axes[0][0],color=color_lin)
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_lin)

y = np.mean(results_svhn['dropout_lin_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_svhn['dropout_lin_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPenDropout', ax=axes[0][0],color=color_dropout_lin)
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_lin)

y = np.mean(results_svhn['nonlin_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_svhn['nonlin_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[0][0], color=color_nonlin)
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_nonlin)

y = np.mean(results_svhn['no_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_svhn['no_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[1][0], color=color_no)
axes[1][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_no)

y = np.mean(results_svhn['dropout_no_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_svhn['dropout_no_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPenDropout', ax=axes[1][0], color=color_dropout_no)
axes[1][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_dropout_no)

y = np.mean(results_svhn['bin_enc']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_svhn['bin_enc']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[2][0], color=color_bin)
axes[2][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_bin)

y = np.mean(results_svhn['512_bin_enc']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_svhn['512_bin_enc']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='WideBinEnc', ax=axes[2][0], color=color_bin_wide)
axes[2][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_bin_wide)


y = np.mean(results_10['lin_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_10['lin_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', ax=axes[0][1],color=color_lin)
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_lin)

y = np.mean(results_10['dropout_lin_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_10['dropout_lin_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPenDropout', ax=axes[0][1],color=color_dropout_lin)
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_lin)

y = np.mean(results_10['nonlin_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_10['nonlin_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[0][1], color=color_nonlin)
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_nonlin)

y = np.mean(results_10['no_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_10['no_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[1][1], color=color_no)
axes[1][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_no)

y = np.mean(results_10['dropout_no_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_10['dropout_no_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPenDropout', ax=axes[1][1], color=color_dropout_no)
axes[1][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_dropout_no)

y = np.mean(results_10['bin_enc']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_10['bin_enc']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[2][1], color=color_bin)
axes[2][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_bin)

y = np.mean(results_10['512_bin_enc']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_10['512_bin_enc']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='WideBinEnc', ax=axes[2][1], color=color_bin_wide)
axes[2][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_bin_wide)

y = np.mean(results_100['lin_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_100['lin_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', ax=axes[0][2],color=color_lin)
axes[0][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_lin)

y = np.mean(results_100['dropout_lin_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_100['dropout_lin_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPenDropout', ax=axes[0][2],color=color_dropout_lin)
axes[0][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_lin)

y = np.mean(results_100['nonlin_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_100['nonlin_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[0][2], color=color_nonlin)
axes[0][2].fill_between(x, y - std, y + std, alpha=alpha, color=color_nonlin)

y = np.mean(results_100['no_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_100['no_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[1][2], color=color_no)
axes[1][2].fill_between(x, y - std, y + std, alpha=alpha, color=color_no)

y = np.mean(results_100['dropout_no_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_100['dropout_no_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPenDropout', ax=axes[1][2], color=color_dropout_no)
axes[1][2].fill_between(x, y - std, y + std, alpha=alpha, color=color_dropout_no)

y = np.mean(results_100['bin_enc']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_100['bin_enc']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[2][2], color=color_bin)
axes[2][2].fill_between(x, y - std, y + std, alpha=alpha, color=color_bin)

y = np.mean(results_100['512_bin_enc']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_100['512_bin_enc']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='WideBinEnc', ax=axes[2][2], color=color_bin_wide)
axes[2][2].fill_between(x, y - std, y + std, alpha=alpha, color=color_bin_wide)


axes[0][0].axvline(x=convergence_svhn['lin_pen'], color=color_lin, linestyle='--',)
axes[0][0].axvline(x=convergence_svhn['dropout_lin_pen'], color=color_dropout_lin, linestyle='-.',)
axes[0][0].axvline(x=convergence_svhn['nonlin_pen'], color=color_nonlin, linestyle=':',)

axes[0][1].axvline(x=convergence_10['lin_pen'], color=color_lin, linestyle='--',)
axes[0][1].axvline(x=convergence_10['dropout_lin_pen'], color=color_dropout_lin, linestyle='-.',)
axes[0][1].axvline(x=convergence_10['nonlin_pen'], color=color_nonlin, linestyle=':',)

axes[0][2].axvline(x=convergence_100['lin_pen'], color=color_lin, linestyle='--',)
axes[0][2].axvline(x=convergence_100['dropout_lin_pen'], color=color_dropout_lin, linestyle='-.',)
axes[0][2].axvline(x=convergence_100['nonlin_pen'], color=color_nonlin, linestyle=':',)


axes[1][0].axvline(x=convergence_svhn['no_pen'], color=color_no, linestyle='--',)
axes[1][0].axvline(x=convergence_svhn['dropout_no_pen'], color=color_dropout_no, linestyle='-.',)

axes[1][1].axvline(x=convergence_10['no_pen'], color=color_no, linestyle='--',)
axes[1][1].axvline(x=convergence_10['dropout_no_pen'], color=color_dropout_no, linestyle='-.',)

axes[1][2].axvline(x=convergence_100['no_pen'], color=color_no, linestyle='--',)
axes[1][2].axvline(x=convergence_100['dropout_no_pen'], color=color_dropout_no, linestyle='-.',)



axes[2][0].axvline(x=convergence_svhn['bin_enc'], color=color_bin, linestyle='--',)
axes[2][0].axvline(x=convergence_svhn['512_bin_enc'], color=color_bin_wide, linestyle='--',)

axes[2][1].axvline(x=convergence_10['bin_enc'], color=color_bin, linestyle='--',)
axes[2][1].axvline(x=convergence_10['512_bin_enc'], color=color_bin_wide, linestyle='--',)

axes[2][2].axvline(x=convergence_100['bin_enc'], color=color_bin, linestyle='--',)
axes[2][2].axvline(x=convergence_100['512_bin_enc'], color=color_bin_wide, linestyle='--',)



axes[0][0].set_title('SVHN', size=label_size)
axes[0][1].set_title('CIFAR 10', size=label_size)
axes[0][2].set_title('CIFAR 100', size=label_size)

axes[2][0].set_yscale('log')
axes[2][1].set_yscale('log')
axes[2][2].set_yscale('log')

axes[1][0].set_ylabel('Within-class covariance', size=label_size)

axes[2][0].set_xlabel('Epochs', size=label_size)
axes[2][1].set_xlabel('Epochs', size=label_size)
axes[2][2].set_xlabel('Epochs', size=label_size)



for ax in axes.flat:
    ax.get_legend().remove()
axes[0][0].legend(loc='upper right', fontsize=legend_size)
axes[1][0].legend(loc='upper right', fontsize=legend_size)
axes[2][0].legend(loc='upper right', fontsize=legend_size)

plt.tight_layout()
plt.savefig('./collapse.png')
'o',label='NonlinPen', ax=axes[0], color=color_nonlin)
axes[0].fill_between(x, y - std, y + std, alpha=alpha, color=color_nonlin)

y = np.mean(results_svhn['no_pen']['same_encoding_fraction_train'], axis=1)
std = np.std(results_svhn['no_pen']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[0], color=color_no)
axes[0].fill_between(x, y - std, y + std, alpha=alpha, color=color_no)

y = np.mean(results_svhn['dropout_no_pen']['same_encoding_fraction_train'], axis=1)
std = np.std(results_svhn['dropout_no_pen']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker='x',label='Dropout', ax=axes[0], color=color_dropout)
axes[0].fill_between(x, y - std, y + std, alpha=alpha, color=color_dropout)



y = np.mean(results_10['bin_enc']['same_encoding_fraction_train'], axis=1)
std = np.std(results_10['bin_enc']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[1], color=color_bin)
axes[1].fill_between(x, y - std, y + std, alpha=alpha, color=color_bin)

y = np.mean(results_10['lin_pen']['same_encoding_fraction_train'], axis=1)
std = np.std(results_10['lin_pen']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', color=color_lin, ax=axes[1])
axes[1].fill_between(x, y - std, y + std, alpha=alpha, color=color_lin)

y = np.mean(results_10['nonlin_pen']['same_encoding_fraction_train'], axis=1)
std = np.std(results_10['nonlin_pen']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[1], color=color_nonlin)
axes[1].fill_between(x, y - std, y + std, alpha=alpha, color=color_nonlin)

y = np.mean(results_10['no_pen']['same_encoding_fraction_train'], axis=1)
std = np.std(results_10['no_pen']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[1], color=color_no)
axes[1].fill_between(x, y - std, y + std, alpha=alpha, color=color_no)

y = np.mean(results_10['dropout_no_pen']['same_encoding_fraction_train'], axis=1)
std = np.std(results_10['dropout_no_pen']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker='x',label='Dropout', ax=axes[1], color=color_dropout)
axes[1].fill_between(x, y - std, y + std, alpha=alpha, color=color_dropout)

x = np.arange(logging, epochs+logging,logging)

y = np.mean(results_100['bin_enc']['same_encoding_fraction_train'], axis=1)
std = np.std(results_100['bin_enc']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[2], color=color_bin)
axes[2].fill_between(x, y - std, y + std, alpha=alpha, color=color_bin)

y = np.mean(results_100['lin_pen']['same_encoding_fraction_train'], axis=1)
std = np.std(results_100['lin_pen']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', color=color_lin, ax=axes[2])
axes[2].fill_between(x, y - std, y + std, alpha=alpha, color=color_lin)

y = np.mean(results_100['nonlin_pen']['same_encoding_fraction_train'], axis=1)
std = np.std(results_100['nonlin_pen']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[2], color=color_nonlin)
axes[2].fill_between(x, y - std, y + std, alpha=alpha, color=color_nonlin)

y = np.mean(results_100['no_pen']['same_encoding_fraction_train'], axis=1)
std = np.std(results_100['no_pen']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[2], color=color_no)
axes[2].fill_between(x, y - std, y + std, alpha=alpha, color=color_no)

y = np.mean(results_100['dropout_no_pen']['same_encoding_fraction_train'], axis=1)
std = np.std(results_100['dropout_no_pen']['same_encoding_fraction_train'], axis=1)
sns.lineplot(x=x, y=y, marker='x',label='Dropout', ax=axes[2], color=color_dropout)
axes[2].fill_between(x, y - std, y + std, alpha=alpha, color=color_dropout)


axes[0].set_title('SVHN', size=label_size)
axes[1].set_title('CIFAR 10', size=label_size)
axes[2].set_title('CIFAR 100', size=label_size)

axes[0].set_xlabel('Epochs', size=label_size)
axes[1].set_xlabel('Epochs', size=label_size)
axes[1].set_xlabel('Epochs', size=label_size)
axes[0].set_ylabel('Fraction', size=label_size)

# axes[0].set_yscale('log')
# axes[0].set_xticks(range(20, 201, 20))

axes[0].get_legend().remove()
axes[1].get_legend().remove()


plt.tight_layout()
plt.savefig('./fraction.png')

### Figure 3

In [ ]:
convergence_svhn = {}
convergence_10 = {}
convergence_100 = {}

for model in models:
    convergence_svhn [model] = np.mean(results_svhn[model]['convergence_epoch']) 
    convergence_10 [model] = np.mean(results_10[model]['convergence_epoch'])    
    convergence_100 [model] = np.mean(results_100[model]['convergence_epoch'])

In [ ]:
x = np.arange(logging, epochs+logging,logging)

fig, axes = plt.subplots(3, 3, figsize=(10, 8), sharex=True)

y = np.mean(results_svhn['lin_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_svhn['lin_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', ax=axes[0][0],color=color_lin)
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_lin)

y = np.mean(results_svhn['nonlin_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_svhn['nonlin_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[0][0], color=color_nonlin)
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_nonlin)

y = np.mean(results_svhn['no_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_svhn['no_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[1][0], color=color_no)
axes[1][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_no)

y = np.mean(results_svhn['dropout_no_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_svhn['dropout_no_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='Dropout', ax=axes[1][0], color=color_dropout)
axes[1][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_dropout)

y = np.mean(results_svhn['bin_enc']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_svhn['bin_enc']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[2][0], color=color_bin)
axes[2][0].fill_between(x, y - std, y + std, alpha=alpha, color=color_bin)


y = np.mean(results_10['lin_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_10['lin_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', ax=axes[0][1],color=color_lin)
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_lin)

y = np.mean(results_10['nonlin_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_10['nonlin_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[0][1], color=color_nonlin)
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_nonlin)

y = np.mean(results_10['no_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_10['no_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[1][1], color=color_no)
axes[1][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_no)

y = np.mean(results_10['dropout_no_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_10['dropout_no_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='Dropout', ax=axes[1][1], color=color_dropout)
axes[1][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_dropout)

y = np.mean(results_10['bin_enc']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_10['bin_enc']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[2][1], color=color_bin)
axes[2][1].fill_between(x, y - std, y + std, alpha=alpha, color=color_bin)


y = np.mean(results_100['lin_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_100['lin_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', ax=axes[0][2],color=color_lin)
axes[0][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_lin)

y = np.mean(results_100['nonlin_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_100['nonlin_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[0][2], color=color_nonlin)
axes[0][2].fill_between(x, y - std, y + std, alpha=alpha, color=color_nonlin)

y = np.mean(results_100['no_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_100['no_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[1][2], color=color_no)
axes[1][2].fill_between(x, y - std, y + std, alpha=alpha, color=color_no)

y = np.mean(results_100['dropout_no_pen']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_100['dropout_no_pen']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='Dropout', ax=axes[1][2], color=color_dropout)
axes[1][2].fill_between(x, y - std, y + std, alpha=alpha, color=color_dropout)

y = np.mean(results_100['bin_enc']['collapse_train']['sigma_w'], axis=1)
std = np.std(results_100['bin_enc']['collapse_train']['sigma_w'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[2][2], color=color_bin)
axes[2][2].fill_between(x, y - std, y + std, alpha=alpha, color=color_bin)


axes[0][0].axvline(x=convergence_svhn['lin_pen'], color=color_lin, linestyle='--',)
axes[0][0].axvline(x=convergence_svhn['nonlin_pen'], color=color_nonlin, linestyle='-.',)

axes[0][1].axvline(x=convergence_10['lin_pen'], color=color_lin, linestyle='--',)
axes[0][1].axvline(x=convergence_10['nonlin_pen'], color=color_nonlin, linestyle='-.',)

axes[0][2].axvline(x=convergence_100['lin_pen'], color=color_lin, linestyle='--',)
axes[0][2].axvline(x=convergence_100['nonlin_pen'], color=color_nonlin, linestyle='-.',)

axes[1][0].axvline(x=convergence_svhn['no_pen'], color=color_no, linestyle='--',)
axes[1][0].axvline(x=convergence_svhn['dropout_no_pen'], color=color_dropout, linestyle='-.',)

axes[1][1].axvline(x=convergence_10['no_pen'], color=color_no, linestyle='--',)
axes[1][1].axvline(x=convergence_10['dropout_no_pen'], color=color_dropout, linestyle='-.',)

axes[1][2].axvline(x=convergence_100['no_pen'], color=color_no, linestyle='--',)
axes[1][2].axvline(x=convergence_100['dropout_no_pen'], color=color_dropout, linestyle='-.',)

axes[2][0].axvline(x=convergence_svhn['bin_enc'], color=color_bin, linestyle='--',)

axes[2][1].axvline(x=convergence_10['bin_enc'], color=color_bin, linestyle='--',)

axes[2][2].axvline(x=convergence_100['bin_enc'], color=color_bin, linestyle='--',)


axes[0][0].set_title('SVHN', size=label_size)
axes[0][1].set_title('CIFAR 10', size=label_size)
axes[0][2].set_title('CIFAR 100', size=label_size)

axes[2][0].set_yscale('log')
axes[2][1].set_yscale('log')
axes[2][2].set_yscale('log')

axes[1][0].set_ylabel('Within-class covariance', size=label_size)

axes[2][0].set_xlabel('Epochs', size=label_size)
axes[2][1].set_xlabel('Epochs', size=label_size)
axes[2][2].set_xlabel('Epochs', size=label_size)



plt.savefig('./collapse.png')

plt.tight_layout()

### Table 1

In [ ]:
print('SVHN')
for model in models:
    print(model, ': ', np.around(np.mean(results_svhn[model]['perturbation_score_tpt']),4), np.around(np.std(results_svhn[model]['perturbation_score_tpt']),4))

print('')
print('CIFAR10')
for model in models:
    print(model, ': ', np.around(np.mean(results_10[model]['perturbation_score_tpt']),4), np.around(np.std(results_10[model]['perturbation_score_tpt']),4))

print('')
print('CIFAR100')      
for model in models:
    print(model, ': ', np.around(np.mean(results_100[model]['perturbation_score_tpt']),4), np.around(np.std(results_100[model]['perturbation_score_tpt']),4))


In [ ]:
print('CIFAR10')
for model in models:
    print(model)
    print('test: ', np.mean(results_10[model]['accuracy_test'], axis=1)[-1], np.std(results_10[model]['accuracy_test'], axis=1)[-1])
    print('train: ', np.mean(results_10[model]['accuracy_train'], axis=1)[-1], np.std(results_10[model]['accuracy_train'], axis=1)[-1])

print('')
print('CIFAR100')      
for model in models:
    print(model)
    print('test: ', np.mean(results_100[model]['accuracy_test'], axis=1)[-1], np.std(results_100[model]['accuracy_test'], axis=1)[-1])
    print('train: ', np.mean(results_100[model]['accuracy_train'], axis=1)[-1], np.std(results_100[model]['accuracy_train'], axis=1)[-1])
        
print('')
print('SVHN')
for model in models:
    print(model)
    print('test: ', np.mean(results_svhn[model]['accuracy_test'], axis=1)[-1], np.std(results_svhn[model]['accuracy_test'], axis=1)[-1])
    print('train: ', np.mean(results_svhn[model]['accuracy_train'], axis=1)[-1], np.std(results_svhn[model]['accuracy_train'], axis=1)[-1])
    

In [ ]:
print('CIFAR10')
for model in models:
    print(model, ': ', np.around(np.mean(results_10[model]['mahalanobis_score_tpt']['auroc']),4), np.around(np.std(results_10[model]['mahalanobis_score_tpt']['auroc']),4))

print('')
print('CIFAR100')      
for model in models:
    print(model, ': ', np.around(np.mean(results_100[model]['mahalanobis_score_tpt']['auroc']),4), np.around(np.std(results_100[model]['mahalanobis_score_tpt']['auroc']),4))

print('')
print('SVHN')      
for model in models:
    print(model, ': ', np.around(np.mean(results_svhn[model]['mahalanobis_score_tpt']['auroc']),4), np.around(np.std(results_svhn[model]['mahalanobis_score_tpt']['auroc']),4))


# Appendix

In [ ]:
x = np.arange(logging, epochs+logging,logging)

fig, axes = plt.subplots(4, 3, figsize=(14, 10), sharex=True)



y = np.mean(results_svhn['bin_enc']['collapse_train']['within_class_variation_weighted'], axis=1)
std = np.std(results_svhn['bin_enc']['collapse_train']['within_class_variation_weighted'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[0][0],color=color_bin)
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin)

y = np.mean(results_svhn['512_bin_enc']['collapse_train']['within_class_variation_weighted'], axis=1)
std = np.std(results_svhn['512_bin_enc']['collapse_train']['within_class_variation_weighted'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEncWide', ax=axes[0][0],color=color_bin_wide)
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin_wide)

y = np.mean(results_svhn['no_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
std = np.std(results_svhn['no_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[0][0],color=color_no)
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_no)

y = np.mean(results_svhn['dropout_no_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
std = np.std(results_svhn['dropout_no_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPenDropout', ax=axes[0][0],color=color_dropout_no)
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_no)

y = np.mean(results_svhn['lin_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
std = np.std(results_svhn['lin_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', ax=axes[0][0],color=color_lin)
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_lin)

y = np.mean(results_svhn['dropout_lin_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
std = np.std(results_svhn['dropout_lin_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPenDropout', ax=axes[0][0],color=color_dropout_lin)
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_lin)

y = np.mean(results_svhn['nonlin_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
std = np.std(results_svhn['nonlin_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[0][0],color=color_nonlin)
axes[0][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_nonlin)



y = np.mean(results_10['bin_enc']['collapse_train']['within_class_variation_weighted'], axis=1)
std = np.std(results_10['bin_enc']['collapse_train']['within_class_variation_weighted'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[0][1],color=color_bin)
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin)

y = np.mean(results_10['512_bin_enc']['collapse_train']['within_class_variation_weighted'], axis=1)
std = np.std(results_10['512_bin_enc']['collapse_train']['within_class_variation_weighted'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEncWide', ax=axes[0][1],color=color_bin_wide)
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin_wide)

y = np.mean(results_10['no_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
std = np.std(results_10['no_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[0][1],color=color_no)
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_no)

y = np.mean(results_10['dropout_no_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
std = np.std(results_10['dropout_no_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPenDropout', ax=axes[0][1],color=color_dropout_no)
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_no)

y = np.mean(results_10['lin_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
std = np.std(results_10['lin_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', ax=axes[0][1],color=color_lin)
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_lin)

y = np.mean(results_10['dropout_lin_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
std = np.std(results_10['dropout_lin_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[0][1],color=color_dropout_lin)
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_lin)

y = np.mean(results_10['nonlin_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
std = np.std(results_10['nonlin_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[0][1],color=color_nonlin)
axes[0][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_nonlin)


y = np.mean(results_100['bin_enc']['collapse_train']['within_class_variation_weighted'], axis=1)
std = np.std(results_100['bin_enc']['collapse_train']['within_class_variation_weighted'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[0][2],color=color_bin)
axes[0][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin)

y = np.mean(results_100['512_bin_enc']['collapse_train']['within_class_variation_weighted'], axis=1)
std = np.std(results_100['512_bin_enc']['collapse_train']['within_class_variation_weighted'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEncWide', ax=axes[0][2],color=color_bin_wide)
axes[0][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin_wide)

y = np.mean(results_100['no_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
std = np.std(results_100['no_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[0][2],color=color_no)
axes[0][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_no)

y = np.mean(results_100['dropout_no_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
std = np.std(results_100['dropout_no_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPenDropout', ax=axes[0][2],color=color_dropout_no)
axes[0][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_no)

y = np.mean(results_100['lin_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
std = np.std(results_100['lin_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', ax=axes[0][2],color=color_lin)
axes[0][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_lin)

y = np.mean(results_100['dropout_lin_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
std = np.std(results_100['dropout_lin_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPenDropout', ax=axes[0][2],color=color_dropout_lin)
axes[0][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_lin)

y = np.mean(results_100['nonlin_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
std = np.std(results_100['nonlin_pen']['collapse_train']['within_class_variation_weighted'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[0][2],color=color_nonlin)
axes[0][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_nonlin)



y = np.mean(results_svhn['bin_enc']['collapse_train']['equiangular'], axis=1)
std = np.std(results_svhn['bin_enc']['collapse_train']['equiangular'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[1][0],color=color_bin)
axes[1][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin)

y = np.mean(results_svhn['512_bin_enc']['collapse_train']['equiangular'], axis=1)
std = np.std(results_svhn['512_bin_enc']['collapse_train']['equiangular'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEncWide', ax=axes[1][0],color=color_bin_wide)
axes[1][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin_wide)

y = np.mean(results_svhn['no_pen']['collapse_train']['equiangular'], axis=1)
std = np.std(results_svhn['no_pen']['collapse_train']['equiangular'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[1][0],color=color_no)
axes[1][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_no)

y = np.mean(results_svhn['dropout_no_pen']['collapse_train']['equiangular'], axis=1)
std = np.std(results_svhn['dropout_no_pen']['collapse_train']['equiangular'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPenDropout', ax=axes[1][0],color=color_dropout_no)
axes[1][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_no)

y = np.mean(results_svhn['lin_pen']['collapse_train']['equiangular'], axis=1)
std = np.std(results_svhn['lin_pen']['collapse_train']['equiangular'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', ax=axes[1][0],color=color_lin)
axes[1][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_lin)

y = np.mean(results_svhn['dropout_lin_pen']['collapse_train']['equiangular'], axis=1)
std = np.std(results_svhn['dropout_lin_pen']['collapse_train']['equiangular'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPenDropout', ax=axes[1][0],color=color_dropout_lin)
axes[1][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_lin)

y = np.mean(results_svhn['nonlin_pen']['collapse_train']['equiangular'], axis=1)
std = np.std(results_svhn['nonlin_pen']['collapse_train']['equiangular'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[1][0],color=color_nonlin)
axes[1][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_nonlin)


y = np.mean(results_10['bin_enc']['collapse_train']['equiangular'], axis=1)
std = np.std(results_10['bin_enc']['collapse_train']['equiangular'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[1][1],color=color_bin)
axes[1][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin)

y = np.mean(results_10['512_bin_enc']['collapse_train']['equiangular'], axis=1)
std = np.std(results_10['512_bin_enc']['collapse_train']['equiangular'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEncWide', ax=axes[1][1],color=color_bin_wide)
axes[1][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin_wide)

y = np.mean(results_10['no_pen']['collapse_train']['equiangular'], axis=1)
std = np.std(results_10['no_pen']['collapse_train']['equiangular'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[1][1],color=color_no)
axes[1][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_no)

y = np.mean(results_10['dropout_no_pen']['collapse_train']['equiangular'], axis=1)
std = np.std(results_10['dropout_no_pen']['collapse_train']['equiangular'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPenDropout', ax=axes[1][1],color=color_dropout_no)
axes[1][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_no)

y = np.mean(results_10['lin_pen']['collapse_train']['equiangular'], axis=1)
std = np.std(results_10['lin_pen']['collapse_train']['equiangular'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', ax=axes[1][1],color=color_lin)
axes[1][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_lin)

y = np.mean(results_10['dropout_lin_pen']['collapse_train']['equiangular'], axis=1)
std = np.std(results_10['dropout_lin_pen']['collapse_train']['equiangular'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPenDropout', ax=axes[1][1],color=color_dropout_lin)
axes[1][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_lin)

y = np.mean(results_10['nonlin_pen']['collapse_train']['equiangular'], axis=1)
std = np.std(results_10['nonlin_pen']['collapse_train']['equiangular'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[1][1],color=color_nonlin)
axes[1][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_nonlin)


y = np.mean(results_100['bin_enc']['collapse_train']['equiangular'], axis=1)
std = np.std(results_100['bin_enc']['collapse_train']['equiangular'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[1][2],color=color_bin)
axes[1][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin)

y = np.mean(results_100['512_bin_enc']['collapse_train']['equiangular'], axis=1)
std = np.std(results_100['512_bin_enc']['collapse_train']['equiangular'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEncWide', ax=axes[1][2],color=color_bin_wide)
axes[1][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin_wide)

y = np.mean(results_100['no_pen']['collapse_train']['equiangular'], axis=1)
std = np.std(results_100['no_pen']['collapse_train']['equiangular'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[1][2],color=color_no)
axes[1][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_no)

y = np.mean(results_100['dropout_no_pen']['collapse_train']['equiangular'], axis=1)
std = np.std(results_100['dropout_no_pen']['collapse_train']['equiangular'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPenDropout', ax=axes[1][2],color=color_dropout_no)
axes[1][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_no)

y = np.mean(results_100['lin_pen']['collapse_train']['equiangular'], axis=1)
std = np.std(results_100['lin_pen']['collapse_train']['equiangular'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', ax=axes[1][2],color=color_lin)
axes[1][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_lin)

y = np.mean(results_100['dropout_lin_pen']['collapse_train']['equiangular'], axis=1)
std = np.std(results_100['dropout_lin_pen']['collapse_train']['equiangular'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPenDropout', ax=axes[1][2],color=color_dropout_lin)
axes[1][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_lin)

y = np.mean(results_100['nonlin_pen']['collapse_train']['equiangular'], axis=1)
std = np.std(results_100['nonlin_pen']['collapse_train']['equiangular'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[1][2],color=color_nonlin)
axes[1][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_nonlin)




y = np.mean(results_svhn['bin_enc']['collapse_train']['maxangle'], axis=1)
std = np.std(results_svhn['bin_enc']['collapse_train']['maxangle'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[2][0],color=color_bin)
axes[2][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin)

y = np.mean(results_svhn['512_bin_enc']['collapse_train']['maxangle'], axis=1)
std = np.std(results_svhn['512_bin_enc']['collapse_train']['maxangle'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEncWide', ax=axes[2][0],color=color_bin_wide)
axes[2][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin_wide)

y = np.mean(results_svhn['no_pen']['collapse_train']['maxangle'], axis=1)
std = np.std(results_svhn['no_pen']['collapse_train']['maxangle'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[2][0],color=color_no)
axes[2][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_no)

y = np.mean(results_svhn['dropout_no_pen']['collapse_train']['maxangle'], axis=1)
std = np.std(results_svhn['dropout_no_pen']['collapse_train']['maxangle'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPenDropout', ax=axes[2][0],color=color_dropout_no)
axes[2][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_no)

y = np.mean(results_svhn['lin_pen']['collapse_train']['maxangle'], axis=1)
std = np.std(results_svhn['lin_pen']['collapse_train']['maxangle'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', ax=axes[2][0],color=color_lin)
axes[2][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_lin)

y = np.mean(results_svhn['dropout_lin_pen']['collapse_train']['maxangle'], axis=1)
std = np.std(results_svhn['dropout_lin_pen']['collapse_train']['maxangle'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPenDropout', ax=axes[2][0],color=color_dropout_lin)
axes[2][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_lin)

y = np.mean(results_svhn['nonlin_pen']['collapse_train']['maxangle'], axis=1)
std = np.std(results_svhn['nonlin_pen']['collapse_train']['maxangle'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[2][0],color=color_nonlin)
axes[2][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_nonlin)



y = np.mean(results_10['bin_enc']['collapse_train']['maxangle'], axis=1)
std = np.std(results_10['bin_enc']['collapse_train']['maxangle'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[2][1],color=color_bin)
axes[2][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin)

y = np.mean(results_10['512_bin_enc']['collapse_train']['maxangle'], axis=1)
std = np.std(results_10['512_bin_enc']['collapse_train']['maxangle'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEncWide', ax=axes[2][1],color=color_bin_wide)
axes[2][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin_wide)

y = np.mean(results_10['no_pen']['collapse_train']['maxangle'], axis=1)
std = np.std(results_10['no_pen']['collapse_train']['maxangle'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[2][1],color=color_no)
axes[2][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_no)

y = np.mean(results_10['dropout_no_pen']['collapse_train']['maxangle'], axis=1)
std = np.std(results_10['dropout_no_pen']['collapse_train']['maxangle'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPenDropout', ax=axes[2][1],color=color_dropout_no)
axes[2][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_no)

y = np.mean(results_10['lin_pen']['collapse_train']['maxangle'], axis=1)
std = np.std(results_10['lin_pen']['collapse_train']['maxangle'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', ax=axes[2][1],color=color_lin)
axes[2][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_lin)

y = np.mean(results_10['dropout_lin_pen']['collapse_train']['maxangle'], axis=1)
std = np.std(results_10['dropout_lin_pen']['collapse_train']['maxangle'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPenDropout', ax=axes[2][1],color=color_dropout_lin)
axes[2][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_lin)

y = np.mean(results_10['nonlin_pen']['collapse_train']['maxangle'], axis=1)
std = np.std(results_10['nonlin_pen']['collapse_train']['maxangle'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[2][1],color=color_nonlin)
axes[2][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_nonlin)



y = np.mean(results_100['bin_enc']['collapse_train']['maxangle'], axis=1)
std = np.std(results_100['bin_enc']['collapse_train']['maxangle'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[2][2],color=color_bin)
axes[2][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin)

y = np.mean(results_100['512_bin_enc']['collapse_train']['maxangle'], axis=1)
std = np.std(results_100['512_bin_enc']['collapse_train']['maxangle'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEncWide', ax=axes[2][2],color=color_bin_wide)
axes[2][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin_wide)

y = np.mean(results_100['no_pen']['collapse_train']['maxangle'], axis=1)
std = np.std(results_100['no_pen']['collapse_train']['maxangle'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[2][2],color=color_no)
axes[2][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_no)

y = np.mean(results_100['dropout_no_pen']['collapse_train']['maxangle'], axis=1)
std = np.std(results_100['dropout_no_pen']['collapse_train']['maxangle'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPenDropout', ax=axes[2][2],color=color_dropout_no)
axes[2][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_no)

y = np.mean(results_100['lin_pen']['collapse_train']['maxangle'], axis=1)
std = np.std(results_100['lin_pen']['collapse_train']['maxangle'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', ax=axes[2][2],color=color_lin)
axes[2][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_lin)

y = np.mean(results_100['dropout_lin_pen']['collapse_train']['maxangle'], axis=1)
std = np.std(results_100['dropout_lin_pen']['collapse_train']['maxangle'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPenDropout', ax=axes[2][2],color=color_dropout_lin)
axes[2][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_lin)

y = np.mean(results_100['nonlin_pen']['collapse_train']['maxangle'], axis=1)
std = np.std(results_100['nonlin_pen']['collapse_train']['maxangle'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[2][2],color=color_nonlin)
axes[2][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_nonlin)


y = np.mean(results_svhn['bin_enc']['collapse_train']['equinorm'], axis=1)
std = np.std(results_svhn['bin_enc']['collapse_train']['equinorm'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[3][0],color=color_bin)
axes[3][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin)

y = np.mean(results_svhn['512_bin_enc']['collapse_train']['equinorm'], axis=1)
std = np.std(results_svhn['512_bin_enc']['collapse_train']['equinorm'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEncWide', ax=axes[3][0],color=color_bin_wide)
axes[3][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin_wide)

y = np.mean(results_svhn['no_pen']['collapse_train']['equinorm'], axis=1)
std = np.std(results_svhn['no_pen']['collapse_train']['equinorm'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[3][0],color=color_no)
axes[3][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_no)

y = np.mean(results_svhn['dropout_no_pen']['collapse_train']['equinorm'], axis=1)
std = np.std(results_svhn['dropout_no_pen']['collapse_train']['equinorm'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPenDropout', ax=axes[3][0],color=color_dropout_no)
axes[3][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_no)

y = np.mean(results_svhn['lin_pen']['collapse_train']['equinorm'], axis=1)
std = np.std(results_svhn['lin_pen']['collapse_train']['equinorm'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', ax=axes[3][0],color=color_lin)
axes[3][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_lin)

y = np.mean(results_svhn['dropout_lin_pen']['collapse_train']['equinorm'], axis=1)
std = np.std(results_svhn['dropout_lin_pen']['collapse_train']['equinorm'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPenDropout', ax=axes[3][0],color=color_dropout_lin)
axes[3][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_lin)

y = np.mean(results_svhn['nonlin_pen']['collapse_train']['equinorm'], axis=1)
std = np.std(results_svhn['nonlin_pen']['collapse_train']['equinorm'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[3][0],color=color_nonlin)
axes[3][0].fill_between(x, y - std, y + std, alpha=alpha,color=color_nonlin)


y = np.mean(results_10['bin_enc']['collapse_train']['equinorm'], axis=1)
std = np.std(results_10['bin_enc']['collapse_train']['equinorm'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[3][1],color=color_bin)
axes[3][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin)

y = np.mean(results_10['512_bin_enc']['collapse_train']['equinorm'], axis=1)
std = np.std(results_10['512_bin_enc']['collapse_train']['equinorm'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEncWide', ax=axes[3][1],color=color_bin_wide)
axes[3][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin_wide)

y = np.mean(results_10['no_pen']['collapse_train']['equinorm'], axis=1)
std = np.std(results_10['no_pen']['collapse_train']['equinorm'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[3][1],color=color_no)
axes[3][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_no)

y = np.mean(results_10['dropout_no_pen']['collapse_train']['equinorm'], axis=1)
std = np.std(results_10['dropout_no_pen']['collapse_train']['equinorm'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPenDropout', ax=axes[3][1],color=color_dropout_no)
axes[3][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_no)

y = np.mean(results_10['lin_pen']['collapse_train']['equinorm'], axis=1)
std = np.std(results_10['lin_pen']['collapse_train']['equinorm'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', ax=axes[3][1],color=color_lin)
axes[3][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_lin)

y = np.mean(results_10['dropout_lin_pen']['collapse_train']['equinorm'], axis=1)
std = np.std(results_10['dropout_lin_pen']['collapse_train']['equinorm'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPenDropout', ax=axes[3][1],color=color_dropout_lin)
axes[3][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_lin)

y = np.mean(results_10['nonlin_pen']['collapse_train']['equinorm'], axis=1)
std = np.std(results_10['nonlin_pen']['collapse_train']['equinorm'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[3][1],color=color_nonlin)
axes[3][1].fill_between(x, y - std, y + std, alpha=alpha,color=color_nonlin)



y = np.mean(results_100['bin_enc']['collapse_train']['equinorm'], axis=1)
std = np.std(results_100['bin_enc']['collapse_train']['equinorm'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEnc', ax=axes[3][2],color=color_bin)
axes[3][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin)

y = np.mean(results_100['512_bin_enc']['collapse_train']['equinorm'], axis=1)
std = np.std(results_100['512_bin_enc']['collapse_train']['equinorm'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='BinEncWide', ax=axes[3][2],color=color_bin_wide)
axes[3][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_bin_wide)

y = np.mean(results_100['no_pen']['collapse_train']['equinorm'], axis=1)
std = np.std(results_100['no_pen']['collapse_train']['equinorm'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPen', ax=axes[3][2],color=color_no)
axes[3][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_no)

y = np.mean(results_100['dropout_no_pen']['collapse_train']['equinorm'], axis=1)
std = np.std(results_100['dropout_no_pen']['collapse_train']['equinorm'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NoPenDropout', ax=axes[3][2],color=color_dropout_no)
axes[3][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_no)

y = np.mean(results_100['lin_pen']['collapse_train']['equinorm'], axis=1)
std = np.std(results_100['lin_pen']['collapse_train']['equinorm'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPen', ax=axes[3][2],color=color_lin)
axes[3][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_lin)

y = np.mean(results_100['dropout_lin_pen']['collapse_train']['equinorm'], axis=1)
std = np.std(results_100['dropout_lin_pen']['collapse_train']['equinorm'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='LinPenDropout', ax=axes[3][2],color=color_dropout_lin)
axes[3][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_dropout_lin)

y = np.mean(results_100['nonlin_pen']['collapse_train']['equinorm'], axis=1)
std = np.std(results_100['nonlin_pen']['collapse_train']['equinorm'], axis=1)
sns.lineplot(x=x, y=y, marker='o',label='NonlinPen', ax=axes[3][2],color=color_nonlin)
axes[3][2].fill_between(x, y - std, y + std, alpha=alpha,color=color_nonlin)



axes[0][0].set_yscale('log')
axes[0][1].set_yscale('log')
axes[0][2].set_yscale('log')

axes[0][0].set_title('SVHN', size=label_size)
axes[0][1].set_title('CIFAR 10', size=label_size)
axes[0][2].set_title('CIFAR 100', size=label_size)

axes[3][0].set_xlabel('Epochs', size=label_size)
axes[3][1].set_xlabel('Epochs', size=label_size)
axes[3][2].set_xlabel('Epochs', size=label_size)


axes[0][0].set_ylabel('NC1', size=label_size)
axes[1][0].set_ylabel('Equiangular', size=label_size)
axes[2][0].set_ylabel('Max angle', size=label_size)
axes[3][0].set_ylabel('Equinorm', size=label_size)

for ax in axes.flat:
    ax.get_legend().remove()
    
axes[3][1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.35), shadow=True, ncol=7)

plt.savefig('./nc_metrics.png')
